In [46]:
import numpy as np
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
def sigmoid_derivative(output):
    return output * (1 - output)

In [47]:
# 整数与二进制转化
int2binary = {}
binary_dim = 8

In [48]:
largest_number = pow(2, binary_dim)

In [49]:
binary = np.unpackbits(np.arange(largest_number, dtype=np.uint8).reshape(-1, 1), axis=1)

In [50]:
for i in range(largest_number):
    int2binary[i] = binary[i]

In [51]:
# 网络结构
n_input = 2
hidden1 = 16
n_output = 1
learning_rate = 0.5

In [52]:
# 初始化模型参数
U = np.random.randn(n_input, hidden1)
V = np.random.randn(hidden1, hidden1)
W = np.random.randn(hidden1, n_output)

In [53]:
dU = np.zeros_like(U)
dV = np.zeros_like(V)
dW = np.zeros_like(W)

In [71]:
# 训练次数
n_epochs = 1


for i in range(n_epochs):
    # 生成加法问题，除以2， 防止加法溢出
    a_int = 15
#     a_int = np.random.randint(largest_number / 2)
    a = int2binary[a_int]
    b_int = 7
#     b_int = np.random.randint(largest_number / 2)

    b = int2binary[b_int]
    
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    d = np.zeros_like(c)
    
    # 训练实例
    X = np.array([a, b]).T
    y = np.array([c]).T
    
    loss = 0
    hs = []  #每个步长下的隐含特征 
    hs.append(np.zeros((1, hidden1)))
    os = [] #每个步长的预测值
    
    # forward
    for t in range(binary_dim):
        xt = X[binary_dim - t - 1]  # 反着来
        ht = sigmoid(np.dot(xt, U) + np.dot(hs[-1], V))
        ot = sigmoid(np.dot(ht, W))
        hs.append(ht)
        os.append(ot)
        loss += np.abs(ot - y[binary_dim - t - 1])[0][0]
        
        # predict
        d[binary_dim - t - 1] = np.round(ot)[0][0]
        
        
    #bp
    future_d_ht = np.zeros((1, hidden1))
    
    for t in reversed(range(binary_dim)):
        xt = X[binary_dim - t - 1].reshape(1, -1)
        ht = hs[t + 1]
        ht_prev = hs[t]
        ot = os[t]
        
        #d_loss/d_ot
        d_ot = ot - y[binary_dim - t - 1]
        d_ot_output = sigmoid_derivative(ot) * d_ot
        dW += ht.T.dot(d_ot_output)
        
        d_ht = d_ot_output.dot(W.T) + future_d_ht
        d_ht_output = sigmoid_derivative(ht) * d_ht
        dU += xt.T.dot(d_ht_output)
        dV += ht_prev.T.dot(d_ht_output)
        
        # 更新futured_d_ht
        future_d_ht = d_ht_output.dot(V.T)
    
    # 梯度更新
    U -= learning_rate * dU
    V -= learning_rate * dV
    W -= learning_rate * dW
    
#     重置梯度
    dU *= 0
    dV *= 0
    dW *= 0
    
    if i % 1000 == 0:
        print("loss:" + str(loss))
        print("Pred:" + str(d))
        print("Ture:" + str(c))
        
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")
        

[[0 0]]
[[0 0]]
[[0 0]]
[[0 0]]
[[1 0]]
[[1 1]]
[[1 1]]
[[1 1]]
loss:0.09470051311147264
Pred:[0 0 0 1 0 1 1 0]
Ture:[0 0 0 1 0 1 1 0]
15 + 7 = 22
------------


In [55]:
a = int2binary[10]
b = int2binary[11]